# Imports

In [2]:
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException
import msal
import requests
client = fabric.FabricRestClient()

StatementMeta(, 53be198c-a29e-4e26-a4ef-280d4d527fc2, 3, Finished, Available, Finished)

# Listar capacidades de Fabric

Debemos escoger la capacidad que vamos a utilizar para asignarla al o las áreas de trabajo que se crearán.

In [3]:
fabric.list_capacities()

StatementMeta(, 53be198c-a29e-4e26-a4ef-280d4d527fc2, 4, Submitted, Running, Running)

In [ ]:
projectName:str = "ProjectName"
layers:list[str] = ["01_BRONZE","02_SILVER","03_GOLD"]
capacityId:str = "CapacityId"
medallionInOneWorkspace:bool = False

azureKeyVault:bool = True
azureKeyVaultName:str = "azure-key-vault-name"
SUBSCRIPTION_ID:str = mssparkutils.credentials.getSecret(f'https://{azureKeyVaultName}.vault.azure.net/', '')
TENANT_ID:str = mssparkutils.credentials.getSecret(f'https://{azureKeyVaultName}.vault.azure.net/', '')
CLIENT_ID:str = mssparkutils.credentials.getSecret(f'https://{azureKeyVaultName}.vault.azure.net/', '')
CLIENT_SECRET:str = mssparkutils.credentials.getSecret(f'https://{azureKeyVaultName}.vault.azure.net/', '')

StatementMeta(, 5d838bba-75f7-4d58-9984-610f2c105db1, 31, Finished, Available, Finished)

# Comprobación de variables obligatorias

In [26]:
def is_null_or_empty(value):
    return value is None or (isinstance(value, str) and value.strip() == "") or (isinstance(value, list) and len(value) == 0)

def validar_configuracion_global():
    errores = []

    # Validación general del proyecto
    if is_null_or_empty(projectName):
        errores.append(" - 'projectName' no está definido o está vacío.")
    if is_null_or_empty(layers):
        errores.append(" - 'layers' no está definido o la lista está vacía.")
    if is_null_or_empty(capacityId):
        errores.append(" - 'capacityId' no está definido o está vacío.")

    # Validación de Azure Key Vault si está habilitado
    if azureKeyVault:
        if is_null_or_empty(azureKeyVaultName):
            errores.append(" - 'azureKeyVaultName' no está definido o está vacío.")
        if is_null_or_empty(SUBSCRIPTION_ID):
            errores.append(" - 'SUBSCRIPTION_ID' no está definido o está vacío.")
        if is_null_or_empty(TENANT_ID):
            errores.append(" - 'TENANT_ID' no está definido o está vacío.")
        if is_null_or_empty(CLIENT_ID):
            errores.append(" - 'CLIENT_ID' no está definido o está vacío.")
        if is_null_or_empty(CLIENT_SECRET):
            errores.append(" - 'CLIENT_SECRET' no está definido o está vacío.")

    # Mostrar errores y detener ejecución si los hay
    if errores:
        print("❌ Se encontraron los siguientes problemas de configuración:")
        for error in errores:
            print(error)
        notebookutils.session.stop()

    print("✅ Configuración validada correctamente.")

validar_configuracion_global()

StatementMeta(, 5d838bba-75f7-4d58-9984-610f2c105db1, 28, Finished, Available, Finished)

✅ Configuración validada correctamente.


# Definición de funciones

In [ ]:
def create_workspace(workspace_name:str,capacity_id:str,description:str):
    """
    Kilian Baccaro | datagym.es

    Función para crear un área de trabajo.

    Parámetros:
    - workspace_name (str): Nombre del área de trabajo a crear.
    - capacity_id (str): ID de la capacidad en la que se asignará el área de trabajo.
    - description (str): Descripción del área de trabajo.

    Devuelve:
    - str: ID del área de trabajo creada si la operación es exitosa.
    - None: En caso de error.
    
    """

    try:
        workspaceId = fabric.create_workspace(workspace_name, capacity_id, description)
        print(f"✅ Área de trabajo {workspace_name} creada con éxito:")
        print(f"ℹ️ Workspace ID: {workspaceId}")
        return workspaceId

    except Exception as error:
        print("❌ Error al crear el área de trabajo: ", error)
        return None


def search_workspace(workspace_name: str):
    """
    Kilian Baccaro | datagym.es

    Busca un área de trabajo en Microsoft Fabric por su nombre.

    Parámetros:
    - workspace_name (str): Nombre del área de trabajo a buscar.

    Devuelve:
    - pd.DataFrame: Un DataFrame con la información del área de trabajo si existe.
    - None: Si ocurre un error o no se encuentra el área de trabajo.

    """

    try:
        workspace = fabric.list_workspaces(filter=f"name eq '{workspace_name}'")
        return workspace
    except Exception as error:
        print("❌ Error al buscar el área de trabajo: ", error)
        return None

def list_lakehouses(workspace_id:str):
    """
    Kilian Baccaro | datagym.es

    Lista todos los lakehouses disponibles en un área de trabajo en Microsoft Fabric.

    Esta función utiliza la API de Microsoft Fabric para obtener una lista de todos los lakehouses
    asociados con un área de trabajo específica, identificada por el parámetro `workspace_id`. 
    Si la operación es exitosa, devuelve una lista con los lakehouses. En caso de que ocurra un 
    error durante la operación, captura la excepción y muestra un mensaje de error.

    Parámetros:
    - workspace_id (str): El identificador del área de trabajo en Microsoft Fabric para la cual 
      se desean obtener los lakehouses.

    Devuelve:
    - list: Una lista de los lakehouses disponibles en el área de trabajo. Si ocurre un error, 
      devuelve `None`.

    """

    try:
        lakehouses = notebookutils.lakehouse.list(workspace_id)
        return lakehouses
    except Exception as error:
        print("❌ Error al listar los lakehouses: ", error)
        return None


def create_lakehouse(lakehouse_name: str, workspace_id: str):
    """
    Kilian Baccaro | datagym.es
    
    Crea un nuevo lakehouse en un área de trabajo de Microsoft Fabric.

    Esta función interactúa con la API de Microsoft Fabric para crear un nuevo lakehouse
    dentro de un área de trabajo específica, identificada por el parámetro `workspace_id`. 
    El lakehouse se crea con el nombre especificado por el parámetro `lakehouse_name`, 
    y se pueden personalizar otros parámetros como la descripción y el número máximo de intentos 
    para la creación. Si la operación es exitosa, se devuelve el id del lakehouse creado. 
    En caso de error, la función captura la excepción y muestra un mensaje de error en consola.

    Parámetros:
    - lakehouse_name (str): El nombre que se asignará al nuevo lakehouse.
    - workspace_id (str): El identificador del área de trabajo en Microsoft Fabric donde se creará 
      el lakehouse.

    Devuelve:
    - str: El ID del lakehouse creado en Microsoft Fabric. Si ocurre un error durante la 
      creación, devuelve `None`.

    """

    description = ""
    max_attempts = 10

    try:
        lakehouseId = fabric.create_lakehouse(lakehouse_name, description, max_attempts, workspace_id)
        return lakehouseId
    except Exception as error:
        print(f"❌ Error al crear el lakehouse {lakehouse_name}: ", error)
        return None


def azurekeyvault_authentication():
    """
    Kilian Baccaro | datagym.es

    Autentica una aplicación contra Azure Key Vault utilizando MSAL (Microsoft Authentication Library) 
    con credenciales de cliente.

    Parámetros globales esperados:
    - TENANT_ID (str): ID del tenant de Azure Active Directory.
    - CLIENT_ID (str): ID de la aplicación registrada en Azure AD.
    - CLIENT_SECRET (str): Secreto del cliente configurado en la app de Azure AD.

    Devuelve:
    - str: Token de acceso (access token) válido para acceder a Azure Key Vault.
    - None: Si ocurre un error durante la autenticación.

    """
    
    AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"
    KV_RESOURCE = "https://vault.azure.net/"
    SCOPE = KV_RESOURCE + ".default"

    app = msal.ConfidentialClientApplication(CLIENT_ID, authority=AUTHORITY, client_credential=CLIENT_SECRET)
    token_response = app.acquire_token_for_client(scopes=[SCOPE])

    if "access_token" in token_response:
        access_token = token_response["access_token"]
        print("✅ Autenticación exitosa para Key Vault")
        return access_token
    else:
        print("❌ Error en la autenticación:", token_response)
        return None

def store_workspace_id_secret(workspace_name: str, workspace_id: str, azure_key_vault_name: str, kv_access_token: str):
    """
    Kilian Baccaro | datagym.es

    Almacena el ID de un Workspace como un secreto en Azure Key Vault.

    Parámetros:
    - workspace_name (str): Nombre del área de trabajo.
    - workspace_id (str): ID del Lakehouse a almacenar como secreto.
    - azure_key_vault_name (str): Nombre del Key Vault de Azure.
    - kv_access_token (str): Token de acceso para autenticación contra Key Vault.

    Devuelve:
    - bool: True si el secreto se almacenó correctamente, False en caso contrario.
    """

    secret_name = f"{workspace_name}-workspace-id".replace("_", "-").lower()
    secret_value = workspace_id

    secret_url = f"https://{azure_key_vault_name}.vault.azure.net/secrets/{secret_name}?api-version=7.3"

    headers = {
        "Authorization": f"Bearer {kv_access_token}",
        "Content-Type": "application/json"
    }

    data = {"value": secret_value}

    response = requests.put(secret_url, headers=headers, json=data)

    if response.status_code in [200, 201]:
        print(f"🔑 Secreto '{secret_name}' almacenado correctamente en {azure_key_vault_name}")
        return True
    else:
        print(f"❌ Error al almacenar el secreto '{secret_name}':", response.text)
        return False


def store_lakehouse_id_secret(project_name: str, layer: str, lakehouse_id: str, azure_key_vault_name: str, kv_access_token: str):
    """
    Kilian Baccaro | datagym.es

    Almacena el ID de un Lakehouse como un secreto en Azure Key Vault.

    Parámetros:
    - project_name (str): Nombre del proyecto.
    - layer (str): Capa del Lakehouse (por ejemplo: 'bronze', 'silver', etc.).
    - lakehouse_id (str): ID del Lakehouse a almacenar como secreto.
    - azure_key_vault_name (str): Nombre del Key Vault de Azure.
    - kv_access_token (str): Token de acceso para autenticación contra Key Vault.

    Devuelve:
    - bool: True si el secreto se almacenó correctamente, False en caso contrario.
    """

    secret_name = f"{project_name}-{layer}-lh-id".replace("_", "-").lower()
    secret_value = lakehouse_id

    secret_url = f"https://{azure_key_vault_name}.vault.azure.net/secrets/{secret_name}?api-version=7.3"

    headers = {
        "Authorization": f"Bearer {kv_access_token}",
        "Content-Type": "application/json"
    }

    data = {"value": secret_value}

    response = requests.put(secret_url, headers=headers, json=data)

    if response.status_code in [200, 201]:
        print(f"🔑 Secreto '{secret_name}' almacenado correctamente en {azure_key_vault_name}")
        return True
    else:
        print(f"❌ Error al almacenar el secreto '{secret_name}':", response.text)
        return False


StatementMeta(, 5d838bba-75f7-4d58-9984-610f2c105db1, 29, Finished, Available, Finished)

# Ejecución

In [ ]:
%%time

# Conexión a Azure Key Vault
if azureKeyVault:
    print("ℹ️ Script configurado para almacenar los secretos en Azure Key Vault")
    kv_access_token = azurekeyvault_authentication()
    print("")

# Si la variable es True, generamos todo en un área de trabajo
if (medallionInOneWorkspace):
    """
    ÁREA DE TRABAJO
    
    Se comprueba si existe el área de trabajo. Si existe, obtenemos el workspaceId, sino, creamos el área de trabajo y obtenemos el workspaceId.
    
    """

    print("ℹ️ El script está configurado para crear todo en una área de trabajo")
    print("ℹ️ Inicializando la creación del área de trabajo...")

    # Buscamos si ya existe el área de trabajo
    workspace = search_workspace(projectName)

    if workspace is not None and not workspace.empty:
        print(f"ℹ️ El workspace {projectName} ya existe. Se crearán los objetos sobre esta área de trabajo.")
        print(f"ℹ️ Workspace ID: {workspace.iloc[0]['Id']}")
        print(f"ℹ️ Capacity ID: {workspace.iloc[0]['Capacity Id']}")
        workspace_id = workspace.iloc[0]['Id']
    else:
        print(f"ℹ️ Creando área de trabajo {projectName}...")
        workspace_id = create_workspace(projectName, capacityId, None)
    
    if workspace_id is None:
        notebookutils.session.stop()

    if azureKeyVault and kv_access_token is not None:
        store_workspace_id_secret(projectName, workspace_id, azureKeyVaultName, kv_access_token)

    print("")

    """
    LAKEHOUSE
    
    Se crean tantos lakehouse como capas se hayan definido en la variable 'layers' con la nomenclatura (projectName)_(layer)
    
    """

    print("ℹ️ Inicializando la creación de los lakehouses...")
    lakehousesList = list_lakehouses(workspace_id)

    lakehousesExistentes = [
        {"name": item["displayName"], "id": item["id"]}
        for item in lakehousesList
    ]


    for layer in layers:
        lakehouseName = f"{projectName}_{layer}_lh"

        if lakehouseName not in [lh["name"] for lh in lakehousesExistentes]:
            print(f"ℹ️ Creando lakehouse {lakehouseName}...")
            lakehouseId = create_lakehouse(lakehouseName, workspace_id)
            if lakehouseId is not None:
                print(f"✅ Lakehouse creado con éxito: {lakehouseId}")
                store_lakehouse_id_secret(projectName, layer, lakehouseId, azureKeyVaultName, kv_access_token)
        
        else:
            print(f"ℹ️ El lakehouse {lakehouseName} ya existe.")
            for lakehouse in lakehousesExistentes:
                if lakehouseName == lakehouse['name']:
                    store_lakehouse_id_secret(projectName, layer, lakehouse['id'], azureKeyVaultName, kv_access_token)

        print("")

    print(f"✅ Ejecución finalizada.")

# Si la variable es False, generamos cada capa en un área de trabajo distinta
else:
    """
    ÁREA DE TRABAJO
    
    Se comprueba si existe el área de trabajo. Si existe, obtenemos el workspaceId, sino, creamos el área de trabajo
    y obtenemos el workspaceId.
    
    """

    print("ℹ️ El script está configurado para crear un área de trabajo para cada capa")
    print("ℹ️ Inicializando la creación del área de trabajo...")

    workspaces_df = fabric.list_workspaces()
    workspacesExistentes = workspaces_df[["Name", "Id"]].to_dict(orient="records")

    for layer in layers:
        workspace_name = f"{projectName}_{layer}"
        workspace = search_workspace(workspace_name)

        if workspace is not None and not workspace.empty:
            print(f"ℹ️ El workspace {workspace_name} ya existe. Se crearán los objetos sobre esta área de trabajo.")
            print(f"ℹ️ Workspace ID: {workspace.iloc[0]['Id']}")
            print(f"ℹ️ Capacity ID: {workspace.iloc[0]['Capacity Id']}")
            print("")
            workspace_id = workspace.iloc[0]['Id']
        else:
            print(f"ℹ️ Creando área de trabajo {workspace_name}...")
            workspace_id = create_workspace(workspace_name, capacityId, None)
        
        if workspace_id is None:
            notebookutils.session.stop()
        
        if azureKeyVault and kv_access_token is not None:
            store_workspace_id_secret(workspace_name, workspace_id, azureKeyVaultName, kv_access_token)

        print("")

        """
        LAKEHOUSE
        
        Se crea el lakehouse correspondiente de la capa
        
        """

        print("ℹ️ Inicializando la creación de los lakehouses...")
        lakehousesList = list_lakehouses(workspace_id)

        lakehousesExistentes = [
            {"name": item["displayName"], "id": item["id"]}
            for item in lakehousesList
        ]

        lakehouseName = f"{projectName}_{layer}_lh"

        if lakehouseName not in [lh["name"] for lh in lakehousesExistentes]:
            print(f"ℹ️ Creando lakehouse {lakehouseName}...")
            lakehouseId = create_lakehouse(lakehouseName, workspace_id)
            if lakehouseId is not None:
                print(f"✅ Lakehouse creado con éxito:")
                store_lakehouse_id_secret(workspace_name, layer, lakehouseId, azureKeyVaultName, kv_access_token)
        
        else:
            print(f"ℹ️ El lakehouse {lakehouseName} ya existe.")
            for lakehouse in lakehousesExistentes:
                if lakehouseName == lakehouse['name']:
                    store_lakehouse_id_secret(projectName, layer, lakehouse['id'], azureKeyVaultName, kv_access_token)
        print("")

    print(f"✅ Ejecución finalizada.")



StatementMeta(, 5d838bba-75f7-4d58-9984-610f2c105db1, 32, Finished, Available, Finished)

ℹ️ Script configurado para almacenar los secretos en Azure Key Vault
✅ Autenticación exitosa para Key Vault

ℹ️ El script está configurado para crear un área de trabajo para cada capa
ℹ️ Inicializando la creación del área de trabajo...
ℹ️ Creando área de trabajo DataGym_DEV_01_BRONZE...
✅ Área de trabajo DataGym_DEV_01_BRONZE creada con éxito:
ℹ️ Workspace ID: 24ef2ff7-b97d-4402-a34e-f34752dfcf92
🔑 Secreto 'datagym-dev-01-bronze-workspace-id' almacenado correctamente en kbsfabrickv

ℹ️ Inicializando la creación de los lakehouses...
ℹ️ Creando lakehouse DataGym_DEV_01_BRONZE_lh...
✅ Lakehouse creado con éxito:
🔑 Secreto 'datagym-dev-01-bronze-01-bronze-lh-id' almacenado correctamente en kbsfabrickv

✅ Ejecución finalizada.
ℹ️ Creando área de trabajo DataGym_DEV_02_SILVER...
✅ Área de trabajo DataGym_DEV_02_SILVER creada con éxito:
ℹ️ Workspace ID: 35d1d7db-8c80-48e0-8255-a6054ec8a397
🔑 Secreto 'datagym-dev-02-silver-workspace-id' almacenado correctamente en kbsfabrickv

ℹ️ Inicializando